# 2. OpenAI의 챗 API 기초


## 2.3. 입출력 길이 제한과 요금에 영향을 미치는 "토큰"


### 토큰


In [2]:
import sys
print(sys.executable)

/home/starrypark/llm-dev/.venv/bin/python


In [3]:
import torch
print(torch.cuda.is_available())  # True면 GPU OK

True


In [4]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

Chat
GPT


### 토크나이저와 tiktoken 소개


In [5]:
import tiktoken

text = "LLM을 사용해 멋진 것을 만들기는 쉽지만, 프로덕션에서 사용할 수 있는 것을 만들기는 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

28


## 2.4. Chat Completions API 테스트 환경 준비


### OpenAI API 키 준비


## 2.5. Chat Completions API 실습


### OpenAI 라이브러리


#### 【주의】알려진 오류에 관해

openai 패키지가 의존하는 httpx의 업데이트로 인해, `openai==1.40.6`을 사용하는 부분에서 `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'`라는 오류가 발생하게 되었습니다.

이 오류는 `!pip install httpx==0.27.2`와 같이, httpx의 특정 버전을 설치함으로써 회피할 수 있습니다.

또한, Google Colab에서 위 오류를 한 번 만난 후 `!pip install httpx==0.27.2`와 같이 패키지를 다시 설치한 경우, 다음 중 하나의 작업을 수행해야 합니다.

- Google Colab의 "런타임"에서 "세션 재시작하기"를 실행합니다
- "런타임 연결 해제 및 삭제"를 실행하여 패키지 설치부터 다시 시작합니다


### Chat Completions API 호출


In [11]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-C5jGL4roMnlS8nC7S2XHR6lV3M8kb",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "안녕하세요, 존님! 만나서 반갑습니다. 무엇을 도와드릴까요?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1755481665,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_560af6e559",
  "usage": {
    "completion_tokens": 20,
    "prompt_tokens": 25,
    "total_tokens": 45,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 대화 이력을 고려한 응답 얻기


In [12]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
        {"role": "assistant", "content": "안녕하세요, 존 님! 만나서 반갑습니다. 오늘은 어떤 이야기를 나눌까요?"},
        {"role": "user", "content": "제 이름을 아시나요?"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-C5jGPwsbxsKme0KDM3XU6pVokF38m",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "네, 존 님이라고 하셨습니다! 어떻게 도와드릴까요?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1755481669,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_560af6e559",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 63,
    "total_tokens": 80,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 스트리밍으로 응답 얻기


In [14]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

안녕하세요, 존님! 어떻게 도와드릴까요?

### JSON 모드


In [15]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '인물 목록을 다음 JSON 형식으로 출력해 주세요.\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "옛날 옛적에 할아버지와 할머니가 살고 있었습니다",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["할아버지", "할머니"]}


### Vision(이미지 입력)


In [16]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

해당 이미지는 "LLM 기반 AI 앱 개발"이라는 제목의 책 표지로 보입니다. 표지에는 파란색과 노란색으로 그려진 귀여운 곤충(벌 또는 비슷한 형태)이 여러 개 나타납니다. 책 제목은 한국어로 적혀 있으며, 카테고리나 저자에 대한 정보도 포함되어 있습니다. 전체적으로 밝고 아기자기한 느낌을 주며, 독자를 끌어당기는 요소가 있는 디자인입니다.


### (칼럼) Completions API


In [11]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="안녕하세요! 저는 존이라고 합니다!",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-B9SHYJhpFCee52ckCtuZvaTIt92wZ",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n반가워요, 저도 처음 만나서"
    }
  ],
  "created": 1741593368,
  "model": "gpt-3.5-turbo-instruct:20230824-v2",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}


## 2.6. Function calling


### Function calling 샘플 코드


In [17]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "seoul" in location.lower():
        return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [18]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [19]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "서울 날씨는 어떤가요?"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-C5jKX4KxtwwSXrmBvWGZIvXqyDTpT",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_xoktR34S6Rg40i6X7WOWkL6t",
            "function": {
              "arguments": "{\"location\":\"Seoul, South Korea\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
  "created": 1755481925,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_07871e2ad8",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 80,
    "total_tokens": 99,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "aud

In [20]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [21]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 사용하고 싶은 함수는 여러 개일 수 있으므로 반복문 사용
for tool_call in response_message.tool_calls:
    # 함수를 실행
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 함수 실행 결과를 대화 이력으로 messages에 추가
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Seoul", "temperature": "10", "unit": null}


In [25]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "서울 날씨는 어떤가요?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_VPAL6aDof51Hck9OOyKG6hKo",
        "function": {
          "arguments": "{\"location\":\"Seoul, KR\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_VPAL6aDof51Hck9OOyKG6hKo",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Seoul\", \"temperature\": \"10\", \"unit\": \"celsius\"}"
  }
]


In [26]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-B9T30F1NsCNuPA2bt3s5BUFqk16Gf",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "현재 서울의 기온은 약 10도입니다. 더 구체적인 날씨 정보가 필요하시면 말씀해 주세요.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741596310,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_eb9dce56a8",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 67,
    "total_tokens": 95,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
